## PROMPT
Generate a pipeline for Databricks:

Get Data information from two web addresses:

Space launches: https://api.spacexdata.com/v3/launches

Space rockets: https://api.spacexdata.com/v3/rockets

Combine each launch record with the name of the corresponding rocket.

Send the final result (launches with rocket names) to the web address: https://httpbin.org/post

The script must provide status updates on its progress, report any errors encountered, confirm the outcome of the final data sending step, and measure/report execution times.


In [0]:
import requests
import json
import time

# Define the URLs
launches_url = "https://api.spacexdata.com/v3/launches"
rockets_url = "https://api.spacexdata.com/v3/rockets"
post_url = "https://httpbin.org/post"

def main():
    print("Starting Databricks pipeline...")

    # --- Step 1: Fetch Space Launches Data ---
    start_time = time.time()
    print("Fetching space launches data...")
    try:
        launches_response = requests.get(launches_url)
        launches_response.raise_for_status()  # Raise an exception for bad status codes
        launches_data = launches_response.json()
        launches_fetch_time = time.time() - start_time
        print(f"Successfully fetched {len(launches_data)} launch records in {launches_fetch_time:.2f} seconds.")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching space launches data: {e}")
        return

    # --- Step 2: Fetch Space Rockets Data ---
    start_time = time.time()
    print("Fetching space rockets data...")
    try:
        rockets_response = requests.get(rockets_url)
        rockets_response.raise_for_status()
        rockets_data = rockets_response.json()
        rockets_fetch_time = time.time() - start_time
        print(f"Successfully fetched {len(rockets_data)} rocket records in {rockets_fetch_time:.2f} seconds.")

        # Create a dictionary to map rocket IDs to names
        rockets_map = {rocket['rocket_id']: rocket['rocket_name'] for rocket in rockets_data}
    except requests.exceptions.RequestException as e:
        print(f"Error fetching space rockets data: {e}")
        return

    # --- Step 3: Combine Launch Data with Rocket Names ---
    print("Combining launch data with rocket names...")
    combined_data = []
    for launch in launches_data:
        rocket_id = launch.get('rocket', {}).get('rocket_id')
        rocket_name = rockets_map.get(rocket_id, "Unknown Rocket")
        launch['rocket_name'] = rocket_name
        combined_data.append(launch)
    print(f"Successfully combined data for {len(combined_data)} launches.")

    # --- Step 4: Send Combined Data to Target URL ---
    start_time = time.time()
    print(f"Sending combined data to {post_url}...")
    try:
        post_response = requests.post(post_url, json=combined_data)
        post_response.raise_for_status()
        post_time = time.time() - start_time
        print(f"Successfully sent combined data in {post_time:.2f} seconds. Response status code: {post_response.status_code}")
        try:
            response_json = post_response.json()
            print("Response from the server:")
            print(json.dumps(response_json, indent=2))
        except json.JSONDecodeError:
            print("Could not decode JSON response from the server.")
    except requests.exceptions.RequestException as e:
        print(f"Error sending data to {post_url}: {e}")
        return

    # --- Step 5: Report Overall Execution Time ---
    total_execution_time = time.time() - script_start_time
    print(f"\nPipeline execution finished in {total_execution_time:.2f} seconds.")

if __name__ == "__main__":
    script_start_time = time.time()
    main()
    